# 🔰 Bronze Layer Template

This notebook is a template for ingesting raw data into the Bronze layer of the medallion architecture.
It includes:
- Reading from landing/ingestion path
- Adding ingestion metadata
- Writing to Delta table

In [0]:


# COMMAND ----------
from pyspark.sql import functions as F
from src.notebook_context import get_notebook_context_info
from src.config import ENVIRONMENTS

# COMMAND ----------
# 🧠 Load environment config from context
ctx = get_notebook_context_info()
env = ctx["env_folder"]
config = ENVIRONMENTS[env]
bronze_path = f"abfss://bronze@{config['storage']}.dfs.core.windows.net/"

# COMMAND ----------
# 📥 Read from ingestion path
input_path = f"/mnt/landing/sample_dataset.json"  # Example path (replace as needed)
df_raw = spark.read.option("multiline", "true").json(input_path)

# COMMAND ----------
# 🧾 Add metadata columns
df_bronze = (
    df_raw
    .withColumn("_ingestion_timestamp", F.current_timestamp())
    .withColumn("_source_file", F.input_file_name())
)

# COMMAND ----------
# 💾 Write to Bronze Delta table
output_table = "bronze.sample_dataset"
(
    df_bronze.write
    .mode("overwrite")  # Change to 'append' in production
    .format("delta")
    .saveAsTable(output_table)
)

print(f"✅ Bronze table written: {output_table}")
